In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
from operator import itemgetter

In [31]:
def ssg_shopping_list(word):
    
    base_url = 'http://earlymorning.ssg.com/search.ssg'
    
    params = {'target':'all','query':word}
    resp = requests.get(base_url, params=params)
    soup = BeautifulSoup(resp.text,'lxml')


    ul_tag = soup.find('ul', class_="cunit_thmb_lst")
    items = ul_tag.find_all('li', class_='cunit_t232')

    no_item = {
        '상품명':word,
        '원가격':"0",
        '할인율':"0",
        '최종가격':"0",
        '비고':'해당상품이 없습니다.'}
    
    
    if items != []:
        item_list = []   
        for item in items[:6]:

            item_info = item.find('div',class_="cunit_info")

            #이름        
            if item_info.find('strong', class_='brd') == None:
                item_name = item_info.find('em', class_='tx_ko').text
            else:
                item_name = item_info.find_all('em', class_='tx_ko')[0].text + item_info.find_all('em', class_='tx_ko')[1].text
                    
                    
            if item.find('div',class_="soldout_area") == None:
                #할인가격
                item_price = item_info.find('em', class_='ssg_price').text

                #원가격
                item_org_price = item_info.find('div',class_ = 'org_price')
                if item_org_price ==  None:
                    item_org_price = item_price
                else:
                    item_org_price = item_info.find('div',class_ = 'org_price').find('em', class_='ssg_price').text

                #할인율(에누리, 쿠폰)
                item_sale_percent = item.find('span', class_='di_enuri')
                if item_sale_percent == None:

                    item_sale_percent = item.find('span', class_='di_coupon')
                    if item_sale_percent == None :
                        item_sale_percent = " "
                    else:
                        item_sale_percent = item.find('span', class_='di_coupon').text

                else:
                    item_sale_percent = item.find('span', class_='di_enuri').text



                #비고
                item_plus = item.find('span', class_='di_plus')
                if item_plus == None:
                    item_plus = " "
                else:
                    item_plus = item.find('span', class_='di_plus').text


                #item_list에 넣기
                item_list.append({
                    '상품명':item_name,
                    '원가격':item_org_price,
                    '할인율':item_sale_percent,
                    '최종가격':item_price,
                    '비고':item_plus
                 })
            else:
                    
                soldout_item = {
                    '상품명':item_name,
                    '원가격':"0",
                    '할인율':"0",
                    '최종가격':"0",
                    '비고':'해당상품은 품절입니다.'                   
                }
                item_list.append(soldout_item)
        
      
            
        best_item = item_list[0]
        
#        n = 0
#        best_item = item_list[n]
#        while word not in best_item["상품명"]:
#            n += 1
#            best_item = item_list[n]

#        if n >= 6 :
#            best_item = no_item
        
        
        
    else:
        best_item = no_item
    
        

    return best_item
            
    

In [ ]:
ssg_list = []
user_keyword = input("상품명: ")
while user_keyword != "":    
    ssg_list.append(ssg_shopping_list(user_keyword))
    user_keyword = input("상품명")

data_ssg = pd.DataFrame(ssg_list)
data_ssg["원가격"] = data_ssg["원가격"].str.replace(",",'').astype(int)
data_ssg["최종가격"] = data_ssg["최종가격"].str.replace(",",'').astype(int)
data_ssg.to_excel("data_ssg.xlsx", encoding ="cp949")

org_total = data_ssg["원가격"].sum()
total = data_ssg["최종가격"].sum()

print("원가격:{},총합계:{}".format(org_total, total))
data_ssg

,상품명,원가격,할인율,최종가격,비고
0,삼다수 2L,980,none,980,none
1,햇 당근(1kg/봉),"4,080",none,"4,080",none
2,특대왕 꿀뚝뚝 청송사과 3입 (팩),"16,800",none,"16,800",none


In [338]:
data_ssg["원가격"] = data_ssg["원가격"].str.replace(",",'').astype(int)
data_ssg["최종가격"] = data_ssg["최종가격"].str.replace(",",'').astype(int)
data_ssg

,상품명,원가격,할인율,최종가격,비고
0,삼다수 2L,980,none,980,none
1,햇 당근(1kg/봉),4080,none,4080,none
2,특대왕 꿀뚝뚝 청송사과 3입 (팩),16800,none,16800,none


In [339]:
org_total = data_ssg["원가격"].sum()
total = data_ssg["최종가격"].sum()

org_total, total

(21860, 21860)